In [1]:
# pip install -q -U google-generativeai


Agent test

In [8]:
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

file_path = os.path.join(parent_dir,'noteBookTest', 'DB', 'testDatabase.txt')

data = ""
# Read the file
with open(file_path, 'r') as f:
    data = f.read()

print(data)

d:\LLMProject\coffeeAgentOrder\src


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\LLMProject\\coffeeAgentOrder\\src\\DB\\testDatabase.txt'

In [3]:
# prompt generation
def generatePrompt(DATA: str, USER_ORDER: str) -> str:
    return f"""
    You are a coffee order assistant that maps customer orders to standard menu items.
    Given a customer order, check the menu and provide the exact corresponding output.

    Menu:
    {DATA}

    Input Order: {USER_ORDER}
    Output Format: [menu_number] [size] [menu_item] [short additional notes or extras: e.g. 3/4, how many sugars, add coffee, add chocolate]

    Rules:
    - guess the coffee order output
    - melbourne magic is a double ristretto flat white
    """

In [4]:
def generateJudgePrompt(output: str, DATA: str, USER_ORDER: str) -> str:
    return f"""You are a precise coffee order validator that evaluates the accuracy of coffee order mappings. Your role is to judge whether the mapped order matches the customer's intent.

Menu:
{DATA}

Original Customer Order: {USER_ORDER}
Coffee agent guess: {output}

Instructions:
1. Analyze if the mapped order accurately represents the customer's intent
2. Consider common coffee order variations and aliases
3. Account for regional naming conventions (e.g., Melbourne Magic = Double Ristretto Flat White)
4. Check for specific customizations (size, strength, milk type)

Evaluation Criteria:
- Base drink match (e.g., latte, cappuccino, flat white)
- Drink size if specified
- Coffee strength (single/double shot, ristretto)
- Milk type if specified
- Temperature preference if mentioned
- Additional modifications (extra hot, no foam, etc.)

""" + """Output Format:
{
    "is_valid": boolean,
    "confidence": float (0.0-1.0),
    "explanation": string,
    "suggested_correction": string (if is_valid is false)
}

Example Response:
{
    "is_valid": true,
    "confidence": 0.95,
    "explanation": "Correctly mapped 'magic' to 'Double Ristretto Flat White'",
    "suggested_correction": null
}
"""

In [5]:
import google.generativeai as genai
import os
MODEL = "gemini-2.0-flash-exp"
api_key = os.getenv('GEMINI_API_KEY')
if not api_key:
    raise ValueError("GEMINI_API_KEY environment variable not found")

genai.configure(api_key = api_key)
model = genai.GenerativeModel(MODEL)
judgeModel = genai.GenerativeModel(MODEL)


c:\Users\hello\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# get user order
USER_ORDER = "large hot chocolate with coffee"
PROMPT = generatePrompt(data,USER_ORDER)
response = model.generate_content(PROMPT)
print(response.text)
judgeResponse = judgeModel.generate_content(generateJudgePrompt(response,data,USER_ORDER))
print(judgeResponse.text)

Input Order: large hot chocolate with coffee
Output: [8] [large] [Hot chocolate] [add coffee]

```json
{
    "is_valid": false,
    "confidence": 0.2,
    "explanation": "The customer ordered a 'hot chocolate with coffee', which is not a standard menu item. The output maps the order as a 'hot chocolate' and 'add coffee'. This is an attempt to interpret the request but is not accurate enough. The correct action would be to ask the customer to clarify their order as this is an unusual order.",
    "suggested_correction": "Clarify with the customer, as 'hot chocolate with coffee' is not a standard option. Suggest either a hot chocolate or a coffee and see what they prefer, or find out if there are other items they are interested in."
}
```



In [7]:
# parse the output to only get the product
import re

def extract_number(text):
    match = re.search(r'\d+', text)
    if match:
        return match.group()
    return None

text = response.text
output_id = extract_number(text)
print(output_id)

8
